In [1]:
import cProfile
import io
import pstats

import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial import cKDTree
from scipy.interpolate import griddata

import functions


In [2]:
import laspy
import numpy as np

def extract_ground_building_points(LasData):
    """
    Extract vegetation points based on classification and NDVI threshold.
    
    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - ndvi_threshold (float): The NDVI threshold to classify vegetation.
    
    Returns:
    - veg_points (laspy.LasData): Filtered vegetation points as a new LasData object.
    """
    # Filter points based on classification (vegetation-related classes)
    building_dsm = LasData[(LasData.classification == 2) |  # ground
                                         (LasData.classification == 6)]   # buildings
                
    return building_dsm


def save_vegetation_points_as_las(LasData, output_file, veg_points):
    """
    Extract vegetation points and save them as a new LAS file.
    
    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - output_file (str): Path to the output LAS file.
    - ndvi_threshold (float): NDVI threshold for vegetation points.
    
    Returns:
    - None
    """
    # Extract vegetation points
    
    # Create a new LasData object for the vegetation points
    veg_las_data = laspy.LasData(LasData.header)
    veg_las_data.points = veg_points.points  # Assign the filtered vegetation points

    # Save the vegetation points to a new LAS file
    with laspy.open(output_file, mode="w", header=veg_las_data.header) as writer:
        writer.write_points(veg_las_data.points)
    
    print(f"Vegetation points saved to {output_file}")


# Example usage:
# Load the LAS/LAZ file
file_path = "../data/25DN2_10.LAZ"



In [3]:
with laspy.open(file_path, laz_backend=laspy.LazBackend.Lazrs) as las_file:
    las_data = las_file.read()



In [4]:
# make delauney triangulation less expensive

def extract_ground_building_points_once(LasData):
    """
    Extract ground and building points based on classification.
    
    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    
    Returns:
    - ground_points (laspy.LasData): Filtered ground points.
    - building_points (laspy.LasData): Filtered building points.
    """
    # Filter points based on classification
    
    dsm_points = LasData[(LasData.classification == 2) |  
                                     (LasData.classification == 6)]

    return dsm_points

def process_las_data(LasData, ground_percent_to_keep, building_percent_to_keep):
    
    points = LasData.points
    classifications = LasData.classification

    ground_mask = (classifications == 2)
    building_mask = (classifications == 6)

    num_points = len(points)
    num_ground_points = np.sum(ground_mask)
    num_building_points = np.sum(building_mask)
    
    num_ground_to_keep = int(num_ground_points * ground_percent_to_keep / 100)
    num_building_to_keep = int(num_building_points * building_percent_to_keep / 100)

    ground_indices = np.where(ground_mask)[0]
    building_indices = np.where(building_mask)[0]

    if num_ground_to_keep > 0:
        sampled_ground_indices = np.random.choice(ground_indices, num_ground_to_keep, replace=False)
    else:
        sampled_ground_indices = np.array([])

    if num_building_to_keep > 0:
        sampled_building_indices = np.random.choice(building_indices, num_building_to_keep, replace=False)
    else:
        sampled_building_indices = np.array([])

    sampled_indices = np.concatenate([sampled_ground_indices, sampled_building_indices])
    combined_points = points[sampled_indices]

    new_las_data = laspy.LasData(LasData.header)
    new_las_data.points = combined_points

    return new_las_data


In [5]:
building_dsm_points = extract_ground_building_points_once(las_data)

In [6]:
import startinpy

In [5]:
def interpolate_laplace(delt, grid, grid_center_xy):

    rows, cols = grid.shape
    grid_center_x, grid_center_y = grid_center_xy
    
    grid = delt.interpolate({"method": "laplace"}, grid_center_xy)
     
        
    return grid

In [6]:
def build_dsm(LasData, points, delt, resolution):

    # Get the extents of the point cloud
    min_x, max_x = round(LasData.x.min()), round(LasData.x.max())
    min_y, max_y = round(LasData.y.min()), round(LasData.y.max())
    
    # Define the size of the region (half_size defines the full size as 2 * half_size)
    x_length = max_x - min_x
    y_length = max_y - min_y
    
    # Number of rows and columns based on the extents and resolution
    cols = round(x_length / resolution)
    rows = round(y_length / resolution)
    
    # Initialize the raster grid with NaN values (no data value)
    dsm_raster = np.full((rows, cols), np.nan, dtype=np.float32)

    # Calculate the center coordinates for each grid cell
    grid_center_xy = functions.raster_center_coords(min_x, max_x, min_y, max_y, resolution)

    if len(points) == 0:
        print("step4, vegetation_raster_idw: There are no vegetation points in the current area.")
        return dsm_raster, grid_center_xy

    # Perform IDW interpolation using k-d tree
    interpolate_laplace(delt, dsm_raster, grid_center_xy)

    return dsm_raster, grid_center_xy

In [17]:
save_vegetation_points_as_las(las_data, "../data/output_buildground_las_og", building_dsm_points)

Vegetation points saved to ../data/output_buildground_las_og


In [ ]:
def step4_main(delt,LasData, buildingdsm,resolution=0.5, search_radius=1.0, idw_power=2.0, write_output=True, show_fig=False):

    builddsm_points = extract_ground_building_points(LasData)
    build_data =  build_dsm(LasData, builddsm_points, delt, resolution)
    veg_raster = build_data[0]
    grid_centers = build_data[1]
    top_left_x = grid_centers[0][0, 0] - resolution / 2
    top_left_y = grid_centers[1][0, 0] + resolution / 2
    if write_output:
        transform = functions.create_affine_transform(top_left_x, top_left_y, resolution)
        functions.write_output(builddsm_points, veg_raster, transform, "../data/builddsm.tiff")

In [7]:
dsm_thinned = process_las_data(las_data, 10, 10)


In [26]:
save_vegetation_points_as_las(las_data, "../data/output_buildground_las_thinned", dsm_thinned)

Vegetation points saved to ../data/output_buildground_las_thinned


In [ ]:

points_list = np.array(dsm_thinned)


In [ ]:
points_list.length
building_dsm_points.length

In [ ]:
dt = startinpy.dt
dt.insert(points_list)

In [ ]:
step4_main(dt, las_data, building_dsm_points, resolution=0.5, search_radius=1.0, idw_power=2.0, write_output=True, show_fig=False)